In [1]:
import numpy as np
import pandas as pd
import datetime
import json
import requests
import time
import os

###### Extraemos los datos meteorológicos de cualquier ciudad registrandose en https://openweathermap.org/city para obtener una llavede acceso y poder generar el archivo json correspondiente, por medio de un link similar al utilizado:

In [2]:
puebla = json.loads(requests.get('http://api.openweathermap.org/data/2.5/weather?q=Puebla,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7').text)

In [3]:
puebla

{'coord': {'lon': -98.2, 'lat': 19.04},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 291.15,
  'pressure': 1025,
  'humidity': 59,
  'temp_min': 291.15,
  'temp_max': 291.15},
 'visibility': 9656,
 'wind': {'speed': 1.45, 'deg': 199},
 'clouds': {'all': 75},
 'dt': 1574785579,
 'sys': {'type': 1,
  'id': 7138,
  'country': 'MX',
  'sunrise': 1574772409,
  'sunset': 1574812422},
 'timezone': -21600,
 'id': 3521081,
 'name': 'Puebla',
 'cod': 200}

###### Para analizar la estructura del archivo json importado utilizamos el siguiente comando, de esa forma, sabremos que campos o indices contiene, por lo que podremos imprimir cada uno de forma individual.

In [4]:
list(puebla.keys())

['coord',
 'weather',
 'base',
 'main',
 'visibility',
 'wind',
 'clouds',
 'dt',
 'sys',
 'timezone',
 'id',
 'name',
 'cod']

In [5]:
print('Coordenadas = ', puebla['coord']) 
print('Clima = ', puebla['weather']) 
print('Base = ', puebla['base']) 
print('Principal = ', puebla['main']) 
print('Viento = ', puebla['wind']) 
print('Nubes = ', puebla['clouds']) 
print('DT = ', puebla['dt']) 
print('SYS = ', puebla['sys']) 
print('ID = ', puebla['id']) 
print('Nombre = ', puebla['name']) 
print('Codigo = ', puebla['cod'])

Coordenadas =  {'lon': -98.2, 'lat': 19.04}
Clima =  [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}]
Base =  stations
Principal =  {'temp': 291.15, 'pressure': 1025, 'humidity': 59, 'temp_min': 291.15, 'temp_max': 291.15}
Viento =  {'speed': 1.45, 'deg': 199}
Nubes =  {'all': 75}
DT =  1574785579
SYS =  {'type': 1, 'id': 7138, 'country': 'MX', 'sunrise': 1574772409, 'sunset': 1574812422}
ID =  3521081
Nombre =  Puebla
Codigo =  200


###### Por ejemplo, podemos ver la temperatura actual accediendo mediante los campos main y temp; dado que el archivo json la provee en grados Kelvin, la convertimos a Celsius 


In [6]:
KelvinTemp = puebla['main']['temp']
CelsiusTemp = KelvinTemp - 273.15
print(CelsiusTemp)

18.0


###### Es necesario identificar cuales seran los datos mas relevantes del archivo, para poder implementar tecnicas de analisis de datos. Los datos deben ser extraidos, limpiados o modificados. Una forma conveniente es insertar algunos procedimientos de extracción a una función, en la que definimos los campos que incluira nuestro dataFrame:.
###### Notese que para la temperatura ya se aplica la conversion a Celsius, ademas de que el parametro dt, dado que es dificil de leer, se convierte a un formato datetime, que especificará la fecha y hora de la extraccion de los datos.

In [7]:
def prepare(ciudad,nombreCiudad):    
    temp = [ ]    
    humidity = [ ]    
    pressure = [ ]    
    description = [ ]    
    dt = [ ]    
    wind_speed = [ ]    
    wind_deg = [ ]    
    temp.append(ciudad['main']['temp']-273.15)    
    humidity.append(ciudad['main']['humidity'])    
    pressure.append(ciudad['main']['pressure'])    
    description.append(ciudad['weather'][0]['description'])    
    dt.append(ciudad['dt'])    
    wind_speed.append(ciudad['wind']['speed'])    
    wind_deg.append(ciudad['wind']['deg'])     
    headings = ['Temperatura','Humedad','Presion','Descripcion','dt','Velocidad viento','Grados viento']
    data = [temp,humidity,pressure,description,dt,wind_speed,wind_deg]    
    df = pd.DataFrame(data,index=headings)    
    ciudad = df.T    
    ciudad['Ciudad'] = nombreCiudad    
    ciudad['Dia'] = ciudad['dt'].apply(datetime.datetime.fromtimestamp)    
    return ciudad




###### De esta forma, podemos definir un dataFrame con los datos extraidos del archivo json de la siguiente manera: 

In [8]:
t1 = prepare(puebla,'Puebla') 
t1

,Temperatura,Humedad,Presion,Descripcion,dt,Velocidad viento,Grados viento,Ciudad,Dia
0,18,59,1025,broken clouds,1574785579,1.45,199,Puebla,2019-11-26 10:26:19


###### Una forma de estructurar los datos de diferentes horarios respecto a la misma ciudad seria:

In [9]:
Puebla = t1 
pueblaNow = json.loads(requests.get('http://api.openweathermap.org/data/2.5/weather?q=Puebla,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7').text)
t2 = prepare(pueblaNow,'Puebla') 
Puebla = Puebla.append(t2) 
Puebla

,Temperatura,Humedad,Presion,Descripcion,dt,Velocidad viento,Grados viento,Ciudad,Dia
0,18,59,1025,broken clouds,1574785579,1.45,199,Puebla,2019-11-26 10:26:19
0,18,59,1025,broken clouds,1574785579,1.45,199,Puebla,2019-11-26 10:26:19


###### Para contar con datos completos, asignaremos un ciclo que agregue los datos meteorologicos de la misma ciudad en distintas horas de manera automatica

In [10]:
t = prepare(puebla,'Puebla') 
Puebla2 = t 
start_time = time.time()
timer = 0
hour = 3
lastHour = 0
while(timer <= hour*2):
    timer = time.time() - start_time
    if(int(timer) % hour == 0 and int(timer) != lastHour):
        lastHour = int(timer) 
        pueblaNow = json.loads(requests.get('http://api.openweathermap.org/data/2.5/weather?q=Puebla,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7').text)
        t = prepare(pueblaNow,'Puebla') 
        Puebla2 = Puebla2.append(t) 
        print(t['Dia'],t['Ciudad'])
        

0   2019-11-26 10:26:19
Name: Dia, dtype: datetime64[ns] 0    Puebla
Name: Ciudad, dtype: object
0   2019-11-26 10:26:19
Name: Dia, dtype: datetime64[ns] 0    Puebla
Name: Ciudad, dtype: object


###### La siguiente funcion nos ayudara a ingresar los datos

In [11]:
def capture2(ciudad,nombreCiudad,link):    
    t = prepare(ciudad,nombreCiudad) 
    print(t['Dia'],t['Ciudad'])   
    Met = t 
    start_time = time.time()
    timer = 0
    minute = 60
    hour = minute * 60
    lastHour = 0
    while(timer <= hour*2):
        timer = time.time() - start_time
        if(int(timer) % hour == 0 and int(timer) != lastHour):
            lastHour = int(timer) 
            Now = json.loads(requests.get(link).text)
            t = prepare(Now,nombreCiudad)
            Met.append(t, ignore_index = True) 
            print(t['Dia'],t['Ciudad'])   
    return Met

In [12]:
#barcelona = json.loads(requests.get('http://api.openweathermap.org/data/2.5/weather?q=Barcelona,es&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7').text)
#Barcelona = capture(barcelona,'Barcelona','http://api.openweathermap.org/data/2.5/weather?q=Barcelona,es&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')
#Barcelona
#El anterior solo fue un ejemplo

###### La siguiente implementacion buscaba ejecutar de forma paralela la recoleccion de datos de las 10 ciudades durante 10 horas. Esto presenta dos problemas: el primero es que ejecutar la funcion fork() en python por medio de os no es posible en Windows, por lo que es pertinente la instalacion de mas componentes. El segundo yace en lo impractico que resulta ejecutar este programa, ya que requeriria su ejecucion continua en una compiutadora durante 10 horas, por lo que se recomienda buscar otra aproximacion para algo tan trivial como la recoleccion de 10 filas de datos

def creator(n):
    i = 1
    while(i<n):
        newpid = os.fork()
        if newpid == 0:
            return i
        i += 1
        print(i)
pid = creator(10)
def cities(pid):
    switcher = {
                    1: "PueblaMet = capture(puebla,'Puebla','http://api.openweathermap.org/data/2.5/weather?q=Puebla,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')",
                    2: "TolucaMaet = capture(toluca,'Toluca','http://api.openweathermap.org/data/2.5/weather?q=Toluca,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')",
                    3: "MoreliaMet = capture(morelia,'Morelia','http://api.openweathermap.org/data/2.5/weather?q=Morelia,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')",
                    4: "QueretaroMet = capture(queretaro,'Queretaro','http://api.openweathermap.org/data/2.5/weather?q=Queretaro,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')",
                    5: "MonterreyMet = capture(monterrey,'Monterrey','http://api.openweathermap.org/data/2.5/weather?q=Monterrey,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')",
                    6: "GuadalajaraMet = capture(guadalajara,'Guadalajara','http://api.openweathermap.org/data/2.5/weather?q=Guadalajara,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')",
                    7: "PachucaMet = capture(pachuca,'Pachuca','http://api.openweathermap.org/data/2.5/weather?q=Pachuca,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')",
                    8: "LeonMet = capture(leon,'Leon','http://api.openweathermap.org/data/2.5/weather?q=Leon,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')",
                    9: "ZacatecasMet = capture(zacatecas,'Zacatecas','http://api.openweathermap.org/data/2.5/weather?q=Zacatecas,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')",
                    10: "TijuanaMet = capture(tijuana,'Tijuana','http://api.openweathermap.org/data/2.5/weather?q=Tijuana,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')"
                }
    return switcher.get(argument, "Error") 
argument


###### Ahora definimos una funcion para capturar cada vez que se desee agregar informacion, acoplandola a dataframes existentes

In [13]:
def capture(nombreCiudad,Met,link): 
    Now = json.loads(requests.get(link).text)
    t = prepare(Now,nombreCiudad) 
    print(t['Dia'],t['Ciudad'])   
    Met = Met.append(t, ignore_index = True) 
    return Met

In [14]:
PueblaMet = pd.read_csv('Puebla.csv')
TolucaMet = pd.read_csv('Toluca.csv')
MoreliaMet = pd.read_csv('Morelia.csv')
QueretaroMet = pd.read_csv('Queretaro.csv')
MonterreyMet = pd.read_csv('Monterrey.csv')
GuadalajaraMet = pd.read_csv('Guadalajara.csv')
PachucaMet = pd.read_csv('Pachuca.csv')
LeonMet = pd.read_csv('Leon.csv')
ZacatecasMet = pd.read_csv('Zacatecas.csv')
TijuanaMet = pd.read_csv('Tijuana.csv')

In [15]:
del PueblaMet['Unnamed: 0']
del TolucaMet['Unnamed: 0']
del MoreliaMet['Unnamed: 0']
del QueretaroMet['Unnamed: 0']
del MonterreyMet['Unnamed: 0']
del GuadalajaraMet['Unnamed: 0']
del PachucaMet['Unnamed: 0']
del LeonMet['Unnamed: 0']
del ZacatecasMet['Unnamed: 0']
del TijuanaMet['Unnamed: 0']

###### El siguiente espacio debe ejecutarse a distintas horas, para acoplar a cada dataFrame

In [20]:
PueblaMet = capture('Puebla',PueblaMet,'http://api.openweathermap.org/data/2.5/weather?q=Puebla,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')
TolucaMet = capture('Toluca',TolucaMet,'http://api.openweathermap.org/data/2.5/weather?q=Toluca,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')
MoreliaMet = capture('Morelia',MoreliaMet,'http://api.openweathermap.org/data/2.5/weather?q=Morelia,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')
QueretaroMet = capture('Queretaro',QueretaroMet,'http://api.openweathermap.org/data/2.5/weather?q=Queretaro,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')
MonterreyMet = capture('Monterrey',MonterreyMet,'http://api.openweathermap.org/data/2.5/weather?q=Monterrey,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')
GuadalajaraMet = capture('Guadalajara',GuadalajaraMet,'http://api.openweathermap.org/data/2.5/weather?q=Guadalajara,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')
PachucaMet = capture('Pachuca',PachucaMet,'http://api.openweathermap.org/data/2.5/weather?q=Pachuca,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')
LeonMet = capture('Leon',LeonMet,'http://api.openweathermap.org/data/2.5/weather?q=Leon,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')
ZacatecasMet = capture('Zacatecas',ZacatecasMet,'http://api.openweathermap.org/data/2.5/weather?q=Zacatecas,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')
TijuanaMet = capture('Tijuana',TijuanaMet,'http://api.openweathermap.org/data/2.5/weather?q=Tijuana,mx&APPID=90b9bba2ea05ea2f0aba69cc7d3e86d7')
 

0   2019-11-26 10:40:18
Name: Dia, dtype: datetime64[ns] 0    Puebla
Name: Ciudad, dtype: object
0   2019-11-26 10:42:59
Name: Dia, dtype: datetime64[ns] 0    Toluca
Name: Ciudad, dtype: object
0   2019-11-26 10:43:00
Name: Dia, dtype: datetime64[ns] 0    Morelia
Name: Ciudad, dtype: object
0   2019-11-26 10:43:01
Name: Dia, dtype: datetime64[ns] 0    Queretaro
Name: Ciudad, dtype: object
0   2019-11-26 10:40:27
Name: Dia, dtype: datetime64[ns] 0    Monterrey
Name: Ciudad, dtype: object
0   2019-11-26 10:43:03
Name: Dia, dtype: datetime64[ns] 0    Guadalajara
Name: Ciudad, dtype: object
0   2019-11-26 10:43:04
Name: Dia, dtype: datetime64[ns] 0    Pachuca
Name: Ciudad, dtype: object
0   2019-11-26 10:39:15
Name: Dia, dtype: datetime64[ns] 0    Leon
Name: Ciudad, dtype: object
0   2019-11-26 10:40:21
Name: Dia, dtype: datetime64[ns] 0    Zacatecas
Name: Ciudad, dtype: object
0   2019-11-26 10:41:35
Name: Dia, dtype: datetime64[ns] 0    Tijuana
Name: Ciudad, dtype: object


In [22]:
PueblaMet.to_csv('Puebla.csv')
TolucaMet.to_csv('Toluca.csv')
MoreliaMet.to_csv('Morelia.csv')
QueretaroMet.to_csv('Queretaro.csv')
MonterreyMet.to_csv('Monterrey.csv')
GuadalajaraMet.to_csv('Guadalajara.csv')
PachucaMet.to_csv('Pachuca.csv')
LeonMet.to_csv('Leon.csv')
ZacatecasMet.to_csv('Zacatecas.csv')
TijuanaMet.to_csv('Tijuana.csv')

In [24]:
PueblaMet

,Temperatura,Humedad,Presion,Descripcion,dt,Velocidad viento,Grados viento,Ciudad,Dia
0,15,82,1022,scattered clouds,1574732261,4.1,190,Puebla,2019-11-25 19:37:41
1,13,87,1023,scattered clouds,1574736536,2.6,170,Puebla,2019-11-25 20:48:56
2,14.03,83,1014,clear sky,1574739418,0.71,359,Puebla,2019-11-25 21:36:58
3,14.03,83,1014,clear sky,1574740953,0.71,359,Puebla,2019-11-25 22:02:33
4,13.13,87,1013,few clouds,1574743175,0.44,296,Puebla,2019-11-25 22:39:35
5,13.13,87,1013,few clouds,1574744600,0.44,296,Puebla,2019-11-25 23:03:20
6,13.13,87,1013,few clouds,1574746337,0.44,296,Puebla,2019-11-25 23:32:17
7,13.13,87,1013,few clouds,1574747615,0.44,296,Puebla,2019-11-25 23:53:35
8,13.13,87,1013,few clouds,1574749337,0.44,296,Puebla,2019-11-26 00:22:17
9,10.99,92,1013,clear sky,1574773508,0.71,11,Puebla,2019-11-26 07:05:08
